<a href="https://colab.research.google.com/github/IqbalLx/Coronavirus-in-Indonesia-Analysis/blob/master/Coronavirus_in_Indonesia_Analysis_nightly_ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing the Dataset

To download a file, go to https://kawalcovid19.blob.core.windows.net/viz/statistik_harian.html. In the top right corner of "Kasus Harian" plotting, there is three horizontal dots that contains exporting data. Choose `.csv` and save in your local folder. Run the below cell, and upload that `.csv` file.

In [5]:
from google.colab import files
uploaded = files.upload() #Data from kawalcovid19.id

Saving Kasus Harian-2020-04-21.csv to Kasus Harian-2020-04-21.csv


In [6]:
# verify last date
val_list = [val for val in uploaded.values()][0]
last_date = val_list.decode().split('\n')[-2]
print(last_date)

2020-04-21 00:00:00,7135,375,95,26


In [7]:
filenames = [key for key in uploaded.keys()]
filenames[0]

'Kasus Harian-2020-04-21.csv'

In [0]:
import pandas as pd
import numpy as np
import datetime

import plotly.graph_objs as go
import matplotlib.pyplot as plt

from sklearn.svm import SVR
from scipy.integrate import odeint
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error as mse

In [9]:
df = pd.read_csv(filenames[0])
print(df.shape)
df.head()

(51, 5)


,DT,Kasus (Kumulatif),Kasus Baru,Sembuh (baru),Meninggal (baru)
0,2020-03-02 00:00:00,2,2.0,NaN,NaN
1,2020-03-03 00:00:00,2,NaN,NaN,NaN
2,2020-03-04 00:00:00,2,NaN,NaN,NaN
3,2020-03-05 00:00:00,2,NaN,NaN,NaN
4,2020-03-06 00:00:00,4,2.0,NaN,NaN


In [10]:
df = df.fillna(0)
df.head()

,DT,Kasus (Kumulatif),Kasus Baru,Sembuh (baru),Meninggal (baru)
0,2020-03-02 00:00:00,2,2.0,0.0,0.0
1,2020-03-03 00:00:00,2,0.0,0.0,0.0
2,2020-03-04 00:00:00,2,0.0,0.0,0.0
3,2020-03-05 00:00:00,2,0.0,0.0,0.0
4,2020-03-06 00:00:00,4,2.0,0.0,0.0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DT                 51 non-null     object 
 1   Kasus (Kumulatif)  51 non-null     int64  
 2   Kasus Baru         51 non-null     float64
 3   Sembuh (baru)      51 non-null     float64
 4   Meninggal (baru)   51 non-null     float64
dtypes: float64(3), int64(1), object(1)
memory usage: 2.1+ KB


In [12]:
df.DT = df.DT.str[:10]    # select only year-month-date in datetime column
df.head()

,DT,Kasus (Kumulatif),Kasus Baru,Sembuh (baru),Meninggal (baru)
0,2020-03-02,2,2.0,0.0,0.0
1,2020-03-03,2,0.0,0.0,0.0
2,2020-03-04,2,0.0,0.0,0.0
3,2020-03-05,2,0.0,0.0,0.0
4,2020-03-06,4,2.0,0.0,0.0


In [13]:
df.DT = pd.to_datetime(df.DT, format="%Y-%m-%d")
df.DT.head()

0   2020-03-02
1   2020-03-03
2   2020-03-04
3   2020-03-05
4   2020-03-06
Name: DT, dtype: datetime64[ns]

In [14]:
df.info()    # check that datetime column is still in datetime format

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   DT                 51 non-null     datetime64[ns]
 1   Kasus (Kumulatif)  51 non-null     int64         
 2   Kasus Baru         51 non-null     float64       
 3   Sembuh (baru)      51 non-null     float64       
 4   Meninggal (baru)   51 non-null     float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 2.1 KB


# Visualizing the Dataset

In [0]:
x = np.array(range(len(df.DT)))
y = df['Kasus Baru'].values

x = x.reshape(-1, 1)

In [16]:
model = SVR(kernel='poly')
model.fit(x, y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [0]:
line = model.predict(x)

In [18]:
name = ['New Cases', 'Died', 'Recovered']
bar = [go.Bar(
        x=df.DT,
        y=df[column],
        name=name[i],
    ) for i, column in enumerate(['Kasus Baru', 'Meninggal (baru)', 
                                  'Sembuh (baru)'])]

reg = go.Scatter(
    x=df.DT,
    y=line,
    line={
        'color':'green',
        'dash': 'dash',
        'width': 3
    },
    name='Trendline'
)

bar.append(reg)

layout = {
    'title':{
        'text': 'Coronavirus in Indonesia',
        'x':0.5, #range x axis 0 - 1
    },
    'xaxis':{
        'title': 'Date',
    },
    'yaxis':{
        'title': 'Num of Cases',
    },
    'legend':{
        'orientation': 'h',
        'x': 0.5,
        'y': 1, #y itu 1 diatas, 0 dibawah
        'xanchor': 'center',
    }
}

fig = go.Figure(data=bar, layout=layout)
fig.show()

In [19]:
y_cum = df['Kasus (Kumulatif)']

model_cum = SVR(kernel='poly')
model_cum.fit(x, y_cum)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [0]:
dt = datetime.datetime(2020, 3, 2)
end = datetime.datetime(2020, 12, 30)
step = datetime.timedelta(days=1)

date_series = []

while dt < end:
    date_series.append(dt.strftime('%Y-%m-%d'))
    dt += step

worst_case = date_series[0:100]
x_pred = np.array(range((len(worst_case))))
x_pred = x_pred.reshape(-1, 1)

line_cum = model_cum.predict(x_pred)

In [21]:
kum = [go.Scatter(
    x=df.DT,
    y=df['Kasus (Kumulatif)'],
    line={
        'color':'green',
        'width': 3
    },
    #mode='lines+markers',
    mode='markers',
    name='Total Cases'
)]

reg_cum = go.Scatter(
    x=worst_case,
    y=line_cum,
    line={
        'color':'orange',
        'dash': 'dash',
        'width': 1
    },
    name='exp. regres.'
)

kum.append(reg_cum)

layout2 = {
    'title':{
        'text': 'Total Cases Coronavirus in Indonesia',
        'x':0.5,  # this position is in unit scale
    },
    'xaxis':{
        'title': 'Date',
    },
    'yaxis':{
        'title': 'Total Cases',
    },
    'legend':{
        'orientation': 'h',
        'x': 0.5,
        'y': 1,  # this position is in unit scale
        'xanchor': 'center',
    }
}

fig2 = go.Figure(kum, layout=layout2)
fig2.show()

# SIR Model Analysis with optimizer

An important rule to remember when using `odeint`, is to make sure that `dt` is small enough and it must not similar to the time interval in the dataset which is `dt` is equal to a day.
That's why we should define a `multplier_or_skip` variable.

In [0]:
def seird(y, t, N, gamma, beta, delta, rho, alpha):
  S, E, I, R, D = y
  dS = -beta(t) * I * S / N
  dE = beta(t) * I * S / N - delta * E
  dR = gamma * (1- alpha) * I
  dD = rho * alpha * I
  dI = delta * E - (dD + dR)
  return dS, dE, dI, dR, dD

Run one of the below cells each time you change which variables
you want to optimize.

## optimize `gamma`, `R_0_start`, `delta`, `rho`, and `alpha` 
(need to choose parameters carefully)

In [154]:
def loss_func(point, data, S0, E0, I0, R0, D0, N, multiplier_or_skip):

  gamma, R_0_start, delta, rho, alpha = point

  k, x0, R_0_end = 0.05, 30, 0.05
  logistic_R_0 = lambda t: (R_0_start-R_0_end) / (1 + np.exp(-k*(-t+x0))) + R_0_end
  beta = lambda t: logistic_R_0(t) * gamma

  N_t = len(data)
  t = np.linspace(0, N_t, N_t*multiplier_or_skip)
  y0 = S0, E0, I0, R0, D0

  seird_res = odeint(seird, y0, t, args=(N, gamma, beta, delta, rho, alpha))
  S, E, I, R, D= seird_res.T


  loss = mse(data, I[::multiplier_or_skip], squared=True)
  #print(round(loss, 2))
  return loss


def train(data, S0, E0, I0, R0, D0, N, multiplier_or_skip):
  optimal = minimize(loss_func, [0.1, 10, 1/14., 1/21., 0.1], 
                     args=(data, S0, E0, I0, R0, D0, N, multiplier_or_skip), 
                     method='L-BFGS-B', bounds=[(1e-8, 1), (1e-8, 1), 
                                                (1e-8, 1), (1e-8, 1),
                                                (1e-8, 1)])
  print(optimal)
  gamma, R_0_start, delta, rho, alpha = optimal.x
  return gamma, R_0_start, delta, rho, alpha


# -----------------------------------------------------------------------------

N = 40_000_000
E0, I0, R0, D0 = 2, 2, 0, 0
S0 = N-(E0+I0+R0+D0) 

N_t = len(date_series)     # this is not the length of available data
multiplier_or_skip = 3     # if N_t = 300 -> len(t) will be = 1000
t = np.linspace(0, N_t, N_t*multiplier_or_skip)
y0 = S0, E0, I0, R0, D0


gamma, R_0_start, delta, rho, alpha = train(df['Kasus (Kumulatif)'], S0, E0, I0, R0, D0, N, multiplier_or_skip)

k = 0.05    # how quickly R_0 declines
x0 = 30     # the x-value of the inflection point i.e. the date of the steepest 
            # decline in R_0,  this could be thought of as the main “lockdown” date)
R_0_end = 0.05

logistic_R_0 = lambda t: (R_0_start-R_0_end) / (1 + np.exp(-k*(-t+x0))) + R_0_end
beta = lambda t: logistic_R_0(t) * gamma

seird_res = odeint(seird, y0, t, args=(N, gamma, beta, delta, rho, alpha))
S, E, I, R, D= seird_res.T

      fun: 37241.75069420016
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([   287.69572964,   -244.74284146, -13631.30868413,   -535.74476624,
          332.48725231])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 630
      nit: 57
   status: 0
  success: True
        x: array([0.66390923, 1.        , 1.        , 0.17419302, 1.        ])


## optimize `gamma`, `R_0_start`, and `alpha`
(need to choose parameters carefully)

In [0]:
def loss_func2(point, data, S0, E0, I0, R0, D0, N, delta, rho, multiplier_or_skip):

  gamma, R_0_start, alpha = point

  k, x0, R_0_end = 0.05, 30, 0.05
  logistic_R_0 = lambda t: (R_0_start-R_0_end) / (1 + np.exp(-k*(-t+x0))) + R_0_end
  beta = lambda t: logistic_R_0(t) * gamma

  N_t = len(data)
  t = np.linspace(0, N_t, N_t*multiplier_or_skip)
  y0 = S0, E0, I0, R0, D0

  seird_res = odeint(seird, y0, t, args=(N, gamma, beta, delta, rho, alpha))
  S, E, I, R, D= seird_res.T

  loss = mse(data, I[::multiplier_or_skip], squared=True)
  #print(round(loss, 2))
  return loss


def train2(data, S0, E0, I0, R0, D0, N, delta, rho, multiplier_or_skip):
  optimal = minimize(loss_func2, [0.1, 10, 0.2], 
                     args=(data, S0, E0, I0, R0, D0, N, delta, rho, multiplier_or_skip), 
                     method='L-BFGS-B', bounds=[(1e-3, 0.5), (10, 20), 
                                                (1e-3, 0.5)])
  print(optimal)
  gamma, R_0_start, alpha = optimal.x
  return gamma, R_0_start, alpha


# -----------------------------------------------------------------------------

N = 40_000_000
E0, I0, R0, D0 = 2, 2, 0, 0
S0 = N-(E0+I0+R0+D0) 

N_t = len(date_series)     # this is not the length of available data
multiplier_or_skip = 3     # if N_t = 300 -> len(t) will be = 1000
t = np.linspace(0, N_t, N_t*multiplier_or_skip)
y0 = S0, E0, I0, R0, D0

delta = 1/14.
rho = 1/21.

gamma, R_0_start, alpha = train2(df['Kasus (Kumulatif)'], S0, E0, I0, R0, D0, N, delta, rho, multiplier_or_skip)

k = 0.05    # how quickly R_0 declines
x0 = 30     # the x-value of the inflection point i.e. the date of the steepest 
            # decline in R_0,  this could be thought of as the main “lockdown” date)
R_0_end = 0.05

logistic_R_0 = lambda t: (R_0_start-R_0_end) / (1 + np.exp(-k*(-t+x0))) + R_0_end
beta = lambda t: logistic_R_0(t) * gamma

seird_res = odeint(seird, y0, t, args=(N, gamma, beta, delta, rho, alpha))
S, E, I, R, D= seird_res.T

## optimize `gamma`, `R_0_start`, `alpha`, `k`, `x0`, and `R_0_end`

(This is not giving the best result. Depend heavily in the choice of the boundary.)

In [181]:
def loss_func3(point, data, S0, E0, I0, R0, D0, N, delta, rho, multiplier_or_skip):

  gamma, R_0_start, alpha, k, x0, R_0_end = point

  logistic_R_0 = lambda t: (R_0_start-R_0_end) / (1 + np.exp(-k*(-t+x0))) + R_0_end
  beta = lambda t: logistic_R_0(t) * gamma

  N_t = len(data)
  t = np.linspace(0, N_t, N_t*multiplier_or_skip)
  y0 = S0, E0, I0, R0, D0

  seird_res = odeint(seird, y0, t, args=(N, gamma, beta, delta, rho, alpha))
  S, E, I, R, D= seird_res.T

  loss = mse(data, I[::multiplier_or_skip], squared=True)
  #print(round(loss, 2))
  return loss


def train3(data, S0, E0, I0, R0, D0, N, delta, rho, multiplier_or_skip):
  optimal = minimize(loss_func3, [0.1, 10, 0.2, 0.05, 30, 0.05], 
                     args=(data, S0, E0, I0, R0, D0, N, delta, rho, multiplier_or_skip), 
                     method='L-BFGS-B', bounds=[(0.09, 0.1), (10, 11), 
                                                (0.1, 0.2), (0.05, 0.06),
                                                (25, 30), (0.05, 0.06)])
  print(optimal)
  return optimal.x


# -----------------------------------------------------------------------------

N = 40_000_000
E0, I0, R0, D0 = 2, 2, 0, 0
S0 = N-(E0+I0+R0+D0) 

N_t = len(date_series)     # this is not the length of available data
multiplier_or_skip = 3     # if N_t = 300 -> len(t) will be = 1000
t = np.linspace(0, N_t, N_t*multiplier_or_skip)
y0 = S0, E0, I0, R0, D0

delta = 1/14.
rho = 1/21.

gamma, R_0_start, alpha, k, x0, R_0_end = train3(df['Kasus (Kumulatif)'], S0, E0, I0, R0, D0, N, delta, rho, multiplier_or_skip)

#k: how quickly R_0 declines
#x0: the x-value of the inflection point i.e. the date of the steepest 
#    decline in R_0,  this could be thought of as the main “lockdown” date)

logistic_R_0 = lambda t: (R_0_start-R_0_end) / (1 + np.exp(-k*(-t+x0))) + R_0_end
beta = lambda t: logistic_R_0(t) * gamma

seird_res = odeint(seird, y0, t, args=(N, gamma, beta, delta, rho, alpha))
S, E, I, R, D= seird_res.T

      fun: 6418118.956626135
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([-50808375.04938245,   -757149.14128184,  -2146866.6382134 ,
       -16019992.43721366,   -176459.17832851,   -517422.98528552])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 14
      nit: 1
   status: 0
  success: True
        x: array([ 0.1 , 11.  ,  0.2 ,  0.06, 30.  ,  0.06])


## Plotting

In [182]:
name = ['Exposed', 'Infected', 'Recovered', 'Death']

data = [go.Scatter(
    x=date_series,
    y=ode[::multiplier_or_skip],
    name=name[i]
) for i, ode in enumerate([E, I, R, D])]


layout3 = {
    'title':{
        'text': 'SEIRD Model for Coronavirus in Indonesia',
        'x':0.5, #range x axis 0 - 1
    },
    'xaxis':{
        'title': 'Date',
    },
    'yaxis':{
        'title': 'Total Cases',
    },
}


fig = go.Figure(data=data, layout=layout3)
fig.show()

In [183]:
data = go.Scatter(
    x=date_series[0:120],
    y=I[::multiplier_or_skip],
    name='Prediction of Total Infected',
    line={
        'dash':'dash'
    }
)

real_data = go.Scatter(
    x=date_series[0:120],
    y=df['Kasus Baru'],
    name='Total Infected from kawalcovid.id',
    mode='markers',
    marker={'symbol':'diamond'}
)

layout3 = {
    'title':{
        'text': 'SEIRD Model for Coronavirus in Indonesia',
        'x':0.5, #range x axis 0 - 1
    },
    'xaxis':{
        'title': 'Date',
        
    },
    'yaxis':{
        'title': 'Total Infected',
        #'range': [-10, 3_000],
    },
}

fig = go.Figure(data=[data, real_data], layout=layout3)
fig.show()